# Year 2023 Day 11


In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from pyobsplot import Plot  # , js # can be used to pass js expr as str

from advent_of_code.visualization.observable_plot import visualize_puzzle_input_202311
from advent_of_code.y_2023.problem_202311 import (
    AdventOfCodeProblem202311,
    create_coord_array,
)

Note: the following logic can be reused for 202324 (show the vectors aligning progressively)

In [ ]:
problem = AdventOfCodeProblem202311()
problem

In [ ]:
puzzle_input = problem.parse_input_text_file()
puzzle_input

In [ ]:
visualize_puzzle_input_202311(puzzle_input)

In [ ]:
visualize_puzzle_input_202311(puzzle_input, with_rules=True)

Let's count how many nodes there are in this graph:

In [ ]:
coord_array = create_coord_array(puzzle_input)
node_count = coord_array["z"].size
node_count

Now we can calculate the number of edges in the related [Complete graph](https://en.wikipedia.org/wiki/Complete_graph), where every pair of node is connected:

In [ ]:
n = node_count
edge_count = n * (n - 1) // 2  # 2 among n
edge_count

The amount is very high, hence in the visualisation, only a fraction is shown

In [ ]:
visualize_puzzle_input_202311(
    puzzle_input,
    with_rules=True,
    with_graph=True,
    coord_array=coord_array,
    edge_density=0.005,
)

In [ ]:
visualize_puzzle_input_202311(
    puzzle_input,
    with_rules=True,
    with_graph=True,
    coord_array=coord_array,
    edge_density=0.03,
)

We can visually see that there are a lot of distances to compute. The plot is filled with edges. Here is an edge of density of 0.1, we can see that the plot is almost filled with the drawing color:

In [ ]:
visualize_puzzle_input_202311(
    puzzle_input,
    with_rules=True,
    with_graph=True,
    coord_array=coord_array,
    edge_density=0.1,
)

The key to expansion is to assign to every chunk a node (it creates a [Lattice graph](https://en.wikipedia.org/wiki/Lattice_graph)).
Then the same logic can be applied on chunks instead of stars, and the distance
becomes the sum of both adjacency matrix, with the chunk one multiplied by
the expansion coefficient

In [ ]:
visualize_puzzle_input_202311(
    puzzle_input,
    with_rules=True,
    with_chunk_graph=True,
)